<a href="https://colab.research.google.com/github/sophielouie/beer-recommendation-system/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import spatial
import statistics as stats
import math
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from networkx.algorithms.dag import topological_sort
import networkx as nx
import plotly.graph_objects as go
from node2vec import Node2Vec
import pickle
#import Node2Vec and Hybrid Notebooks
import node2vec_module
from node2vec_module import node2vec, clean_beer_reviews
import hybridmodel as h
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
import hybridmodel
from hybridmodel import clean_beer_reviews
import pandas as pd

we want to import functions into our gui file

# app interface

In [4]:
# paragraph describing each model

In [5]:
BEER_DF = clean_beer_reviews()

KeyboardInterrupt: 

In [ ]:
# make a fucntion that returns a list of beers
def make_beer_list():
    return BEER_DF["Beer Name"].unique()


# list of user, beer combination
def make_beer_user_list(user):
    return BEER_DF.loc[BEER_DF["review_profilename"] == user]["Beer Name"]

# list of users to choose from
def make_user_list():
    return BEER_DF['review_profilename'].unique()

In [ ]:
# MDOEL TYPE DROPDOWN TO USER/BEER OPTIONS
def model_dropdown_eventhandler(change):
    determine(model_dropdown.value)

option_list = ["Hybrid", "Node2Vec"]

user = make_user_list()[:100]
beer = make_beer_list()

def determine(x):
    third_dropdown.layout.visibility = "hidden"
    sec_dropdown.index = None
    if x == "Hybrid":
        sec_dropdown.options = user
    else:
        sec_dropdown.options = beer
        
# HYBRID MODEL USER CHOISE TO CORRESPONDING BEER OPTIONS
def beer_dropdown_eventhandler(change):
    determine_beer(sec_dropdown.value)

def determine_beer(x):
    third_dropdown.index = None
    beer_user = make_beer_user_list(x)
    if len(beer_user) > 0:
        third_dropdown.options = beer_user
        third_dropdown.layout.visibility = "visible"
    else: 
        third_dropdown.layout.visibility = "hidden"

model_dropdown = Dropdown(description="Select a model", options=option_list,
            style = {'description_width': 'initial'}, value=None)
#This will be users for when hybrid is selected, beers for when Node2Vec is selected
sec_dropdown = Dropdown(description="Choose one",
            style = {'description_width': 'initial'})
model_dropdown.observe(model_dropdown_eventhandler, names='value')
#This will only appear when Hybrid is selected and contains beer list
third_dropdown = Dropdown(description="Choose one", layout = {"visibility" : "hidden"},
            style = {'description_width': 'initial'})
sec_dropdown.observe(beer_dropdown_eventhandler, names='value')
num_recs = widgets.IntSlider(
            value=5,
            min=0,
            max=10,
            step=1,
            description='Number of recommendations:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            layout=widgets.Layout(width='50%', height='40px'),
            style = {'description_width': 'initial'}
        )

display(model_dropdown, sec_dropdown, third_dropdown, num_recs)

In [ ]:
btn = widgets.Button(description='Start')
out = widgets.Output(layout={'border': '1px solid black'})

display(btn, out)

def btn_eventhandler(obj):
    out.clear_output()
    
    if model_dropdown.value == 'Hybrid':
        if sec_dropdown.value and third_dropdown.value and num_recs:
            with out:
                print('running')
                print(h.hybrid_model(sec_dropdown.value, third_dropdown.value, num_recs.value))
    
    elif model_dropdown.value == 'Node2Vec':
        if sec_dropdown.value and num_recs:
            with out:
                print('running')
                print(node2vec(sec_dropdown.value, num_recs.value))
    
btn.on_click(btn_eventhandler)